In [1]:
import os
import cv2 
import numpy as np
import mediapipe as mp
import time
from tensorflow import keras
import tensorflow
# import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard
mp_holistic = mp.solutions.holistic # Holistic model to detect keypoints
mp_drawing = mp.solutions.drawing_utils # Drawing utilities)
print(keras.__version__)
# print(keras2.__version__)

2022-10-30 11:51:34.517214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.10.0


In [2]:
actions = np.array(["Left_punch","kiss","no_action","Right_punch","Wakanda"])
augmentations = ["CW Rotate 15","CCW Rotate 15","Left","Right","Zoom In","Zoom Out"]
label_map = {label:idx for idx,label in enumerate(actions)}
label_map

{'Left_punch': 0, 'kiss': 1, 'no_action': 2, 'Right_punch': 3, 'Wakanda': 4}

In [2]:
def keypointsToNumPy(results):
    pose_landmarks = np.array([[l.x,l.y,l.z,l.visibility] for l in results.pose_landmarks.landmark])
    lefthand_landmarks = np.array([[l.x,l.y,l.z] for l in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks and len(results.left_hand_landmarks.landmark)>0 else np.zeros(21*3)
    righthand_landmarks = np.array([[l.x,l.y,l.z] for l in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks and len(results.right_hand_landmarks.landmark)>0 else np.zeros(21*3)
    landmarks = np.concatenate([pose_landmarks.flatten(),lefthand_landmarks.flatten(),righthand_landmarks.flatten()])
    return landmarks

In [1]:
model = keras.models.load_model('transformer.h5')
frames = []

NameError: name 'keras' is not defined

In [4]:
#!/usr/bin/env python

import asyncio
import websockets
import json
from attrdict import AttrDict

def keypointsToNumPy(results):
    pose_landmarks = np.array([[l.x,l.y,l.z,l.visibility] for l in results.pose_landmarks.landmark])
    lefthand_landmarks = np.array([[l.x,l.y,l.z] for l in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks and len(results.left_hand_landmarks.landmark)>0 else np.zeros(21*3)
    righthand_landmarks = np.array([[l.x,l.y,l.z] for l in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks and len(results.right_hand_landmarks.landmark)>0 else np.zeros(21*3)
    landmarks = np.concatenate([pose_landmarks.flatten(),lefthand_landmarks.flatten(),righthand_landmarks.flatten()])
    return landmarks


async def echo(websocket):
    frames = []
    async for message in websocket:
        # print(message)
        try:
            y = json.loads(message)
            results = AttrDict({
                "pose_landmarks":{
                    "landmark":[]
                },
                "left_hand_landmarks":{
                    "landmark":[]
                },
                "right_hand_landmarks":{
                    "landmark":[]
                }
            })
            results["pose_landmarks"]["landmark"] = y["poseLandmarks"]
            if "leftHandLandmarks" in y:
                results["left_hand_landmarks"]["landmark"] = y["leftHandLandmarks"]
            if "rightHandLandmarks" in y:
                results["right_hand_landmarks"]["landmark"] = y["rightHandLandmarks"]
            keypoints = keypointsToNumPy(results)
            frames.append(keypoints)
            # just get last 6 frames
            frames = frames[-6:]
            if len(frames) == 6:
                a = np.array(frames)
                # convert it to proper input (1,30,258)
                print(np.expand_dims(frames,axis=0).shape)
                # get the first prediction as we only pass one into it
                r = model.predict(np.expand_dims(frames,axis=0))[0]
                # r[r<0.5] = 0
                # print(actions[np.argmax(r)])
                j = {
                    "action": actions[np.argmax(r)],
                    "accuracy": r
                }
                # await websocket.send(actions[np.argmax(r)] + "," + str(r))
                await websocket.send(json.dumps(j, separators=(',', ':')))
        except Exception as e:
            print(e)
        # await websocket.send(message + "server")

async def main():
    async with websockets.serve(echo, "localhost", 8765):
        await asyncio.Future()  # run forever

await main()